In [1]:
import os
from pathlib import Path
from openai import OpenAI

from segment_finder import SegmentFinder
from segment_rewriter import SegmentRewriter
from utils.pdf_utils import compile_tex_to_pdf, extract_text_from_pdf
from utils.segment_utils import inject_segment_markers, replace_marked_segment

In [2]:
resume_name = 'resume-1'
job_description_name = 'job-description-1'

input_path = Path('input')
output_path = Path('output')
output_path.mkdir(parents = True, exist_ok = True)

resume_path = input_path / f'resumes/{resume_name}.tex'
job_description_path = input_path / f'jobs/{job_description_name}.txt'

In [3]:
def read_text(path):
    return path.read_text(encoding = 'utf-8')

resume_tex_content = read_text(resume_path)
job_description_content = read_text(job_description_path)

print('resume length:', len(resume_tex_content))
print('job description length:', len(job_description_content))

resume length: 8636
job description length: 2989


In [4]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not OPENAI_API_KEY:
    raise Exception('OPENAI_API_KEY not set.')

openai_client = OpenAI(api_key = OPENAI_API_KEY)
openai_model = 'gpt-4o-mini'

In [5]:
segment_finder = SegmentFinder(openai_client, openai_model)
segments = segment_finder.call_segment_finder(resume_tex_content, job_description_content)
marked_resume, invalid_segment_ids = inject_segment_markers(resume_tex_content, segments)

segments = [segment for segment in segments if segment.id not in invalid_segment_ids]

Segment latex not found for segment s10. Make sure the segment finder returns exact snippets from the resume.
Segment latex not found for segment s11. Make sure the segment finder returns exact snippets from the resume.
Segment latex not found for segment s12. Make sure the segment finder returns exact snippets from the resume.


In [6]:
segments

[Segment(id='s1', reason='Lacks specific mention of .NET technologies, which are crucial for the job description.', latex='{Designed and implemented backend services for vehicle telemetry processing using Python and C\\#, handling millions of events per day.}'),
 Segment(id='s2', reason='Missing emphasis on cloud and Azure experience, which is essential for the targeted role.', latex='{Built data ingestion pipelines using Azure Event Hubs and Azure Functions to process real time sensor data.}'),
 Segment(id='s3', reason='Needs to highlight teamwork and collaboration aspects, as these are key in the job description.', latex='{Wrote unit and integration tests, enforced code quality through code reviews and CI pipelines.}'),
 Segment(id='s4', reason='Could emphasize experience with Agile methodologies, which is important for the job position.', latex='{Developed low latency trading infrastructure components in C++ with strict performance and correctness requirements.}'),
 Segment(id='s5',

In [7]:
segment_rewriter = SegmentRewriter(openai_client, openai_model)

rewrites = []
for segment in segments:
    rewrites.append(segment_rewriter.call_segment_rewriter(segment.id, segment.latex, job_description_content))

In [8]:
rewrites

[RewriteResultWithOriginal(segment_id='s1', original_latex='{Designed and implemented backend services for vehicle telemetry processing using Python and C\\#, handling millions of events per day.}', rewritten_latex='{Designed and implemented backend services for scalable vehicle telemetry processing using Python and C\\#, handling millions of events per day while ensuring performance and reliability.}', notes=''),
 RewriteResultWithOriginal(segment_id='s2', original_latex='{Built data ingestion pipelines using Azure Event Hubs and Azure Functions to process real time sensor data.}', rewritten_latex='{Developed data ingestion pipelines using Azure Event Hubs and Azure Functions to process and analyze real-time sensor data, ensuring scalability and responsiveness for subsequent data utilization.}', notes=''),
 RewriteResultWithOriginal(segment_id='s3', original_latex='{Wrote unit and integration tests, enforced code quality through code reviews and CI pipelines.}', rewritten_latex='{Wrot

In [9]:
optimized_resume_tex = marked_resume
for rewrite in rewrites:
    optimized_resume_tex = replace_marked_segment(optimized_resume_tex, rewrite.segment_id, rewrite.rewritten_latex)

In [10]:
optimized_dir = output_path / f'optimized-{resume_name}'
optimized_dir.mkdir(parents = True, exist_ok = True)

optimized_tex_path = optimized_dir / f'optimized-{resume_name}.tex'

optimized_tex_path.write_text(optimized_resume_tex, encoding = 'utf-8')

9175

In [11]:
compile_tex_to_pdf(optimized_tex_path, output_path / f'optimized-{resume_name}')

WindowsPath('output/optimized-resume-1/optimized-resume-1.pdf')